# Installation and Setup


In [ ]:
# Install the Prediction Guard client
! pip install -U predictionguard
! pip install --upgrade predictionguard

In [ ]:
#Import packages
import os
import json
import pandas as pd
from getpass import getpass

# Transcripts
with open('transcripts.json') as f:
  transcripts = json.load(f)

# Test data
test = pd.read_csv('test.csv')

# Set your prediction guard access token (this will be provided to
# you via email if you filled out the kick off call survey)
pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token
os.environ['PREDICTIONGUARD_URL'] = "https://intel.predictionguard.com"
import predictionguard as pg

# How To Create Prompts

In [ ]:
# Create an LLM prompt.
prompt = """### Instruction:
Tell me a joke

### Response:
"""

result = pg.Completion.create(
    model="Nous-Hermes-Llama2-13B",
    prompt=prompt
)

print(json.dumps(
    result,
    sort_keys=True,
    indent=4,
    separators=(',', ': ')
))

print(result["choices"][0]['text']) #text response

In [ ]:
# Create an LLM "prompt" that we will use to extract certain
# information from the transcript.

#using first transcript in the file
prompt = f"""### Instruction:
Extract the patient's condition from the below input.

### Input:
{transcripts['2055']}

### Response:
""".format(transcript=transcripts['2055'])

In [ ]:
# Use Nous-Hermes-Llama2-13B to extract the information.
result = pg.Completion.create(
    model="Nous-Hermes-Llama2-13B",
    prompt=prompt #using prompt the cell above (Extract the patient's condition from the below input.)
)
print(result)
print(result["choices"][0]["text"]) #print result

# Testing Model on Dataset (no augmentation/zero-shot)


In [ ]:
#running model on every prompt

def formatPrompt(q, t):
  p = f"""
  ### Instruction:
  {q}
  ### Input:
  {t}
  ### Response:
  """.format(transcript=t)
  return p

solution_raw = []
i = 0
for index, row in test.iterrows(): #goes through each row of test.csv
  transcript = transcripts[str(row['Transcript'])]
  #print(transcript)
  question = row["Question"] #current prompt
  prompt = formatPrompt(question,transcript) #formatting prompt so model can run it
  response = ""
  try:
    result = pg.Completion.create( #stores result
        model="Nous-Hermes-Llama2-13B",
        prompt=prompt
    )
    response = result["choices"][0]["text"]
  except:
    response = "Error"

  solution_raw.append([ # adding results to solution csv
      row['Id'],
      response
  ])
  if i>20:break
  i+=1

  print(response)
  print()

solution = pd.DataFrame(solution_raw, columns=["Id", "Text"]) #turning into panda dataframe
solution.head()

# Basic Prompting


In [ ]:
#running model on every prompt

def formatPrompt(t, q):
  p = f"""
  ### Instruction:
  {t}
  ### Input:
  {q}
  ### Response:
  """.format(transcript=t)
  return p

format = {
    "What is the patient's name?":"The name is...",
    "What is the patient's age?":"The age is...",
    "What is the patient's condition?":"The condition is...",
    "What symptoms is the patient experiencing?":"The patient is experiencing...",
    "What precautions did the doctor advise?":"The doctor advises...",
    "What drug did the doctor prescribe?":"The doctor prescribed..."
}

examples = {
    "What is the patient's name?":
      """
      Example: The name is John Smith.
      Example: The name is Kevin Bacon.
      Example: The name is Jeffery Montblanc.
      Example: The name is Edgar Allen Poe.
      Example: The name is Selena Gomez.
      Example: The name is Ajeet Singh.
      Example: The name is Kevin Nguyen.
      Example: The name is Maria Kim.
      Example: The name is Hellen Keller.
      """,
    "What is the patient's age?":
      """
      Example: 25-years-old
      Example: 3-years-old
      Example: 98-years-old
      Example: 72-years-old
      Example: 37-years-old
      Example: 44-years-old
      Example: 51-years-old
      Example: 29-years-old
      Example: 63-years-old
      Example: 86-years-old
      """,
    "What is the patient's condition?":
      """
      Example: The condition is a fungal infection.
      Example: The condition is hypothermia.
      Example: The condition is excessive body odor.
      Example: The condition is diabetes.
      Example: The condition is fractured tailbone.
      Example: The condition is food poisoning.
      Example: The condition is pneumonia.
      Example: The condition is hives.
      Example: The condition is a stroke.
      Example: The condition is cancer.
      """,
    "What symptoms is the patient experiencing?":
      """
      Example: The symptoms are itchiness, swelling, and pain around the affected area.
      Example: The symptoms are unable to get up.
      Example: The symptoms are coughing, loss of taste, and muscle fatigue.
      Example: The symptoms are muscle soreness and elbow pain.
      Example: The symptoms are stomach aches, vomitting, and chills.
      Example: The symptoms are diarrhea, fever, and stomach pain.
      Example: The symptoms are hard time sleeping and cold sweats.
      Example: The symptoms are nausea and dizziness.
      Example: The symptoms are chest pain and heart burn.
      Example: The symptoms are shortness of breathe, dehydration, and overheating.

      """,
    "What precautions did the doctor advise?":
      """
      Example: The doctor advised washing the wound, applying ointment, and a bandage.
      Example: The doctor advised drinking more water.
      Example: The doctor advised eating less food and walking more.
      Example: The doctor advised to not run for three weeks.
      Example: The doctor advised reducing cholesterol intake and eating more fiber.
      Example: The doctor advised getting more sleep, drinking less caffeine, and taking vitamin C.
      Example: The doctor advised doing more stretches and swimming.
      Example: The doctor advised watching less TV and getting 60 minutes of exercise.
      Example: The doctor advised to see a physical therapist.
      Example: The doctor advised getting tested for allergies and staying away from lactose.
      """,
    "What drug did the doctor prescribe?":
      """
      Example: The doctor prescribed antibiotics.
      Example: The doctor prescribed motrin.
      Example: The doctor prescribed lisinopril.
      Example: The doctor prescribed levothyroxine.
      Example: The doctor prescribed atorvastatin.
      Example: The doctor prescribed metformin.
      Example: The doctor prescribed simvastatin.
      Example: The doctor prescribed albuterol.
      Example: The doctor prescribed amlodipine.
      Example: The doctor prescribed nothing.

      """
}

solution_raw = []
i = 0
for index, row in test.iterrows(): #goes through each row of test.csv
  transcript = transcripts[str(row['Transcript'])]
  question = row["Question"] #current prompt
  instruction = "Answer the question below based off the transript. Answer with the same format as the examples given.\n"
  prompt = formatPrompt(instruction + examples[question], "Transcript:\n" + transcript + "\n\nQuestion:" + question) #formatting prompt so model can run it
  print(prompt)
  response = ""
  try:
    result = pg.Completion.create( #stores result
        model="Nous-Hermes-Llama2-13B",
        prompt=prompt
    )
    response = result["choices"][0]["text"]
  except:
    response = "Error"

  solution_raw.append([ # adding results to solution csv
      row['Id'],
      response
  ])
  if i>20:break
  i+=1

  print(response)
  print()

solution = pd.DataFrame(solution_raw, columns=["Id", "Text"]) #turning into panda dataframe
solution.head()

# Save to File


In [ ]:
solution.to_csv("sample_submission.csv", index=False) #saving to file